# Análise da Correlação de Métricas de Países com sua Emissão de Carbono

## Alunos

- Bruno de Marco Appolonio - 195036
- João Vitor Vendemiato Fatoretto - 199944
- Lucas Costa de Oliveira - 182410
- Natan Beltrão da Cunha Pevidor Carvalho - 184972

## Introdução

Neste projeto, desenvolvemos uma solução em machine learning para um dos problemas relacionados com os [Objetivos de Desenvolvimento Sustentável](https://brasil.un.org/pt-br/sdgs) da ONU. Optamos por criar uma solução relacionado predominantemente com o objetivo 13 (Ação contra a mudança global do clima), porém relacionada também com outros objetivos.

Buscamos analisar a correlação entre diversas métricas e a emissão de carbono anual de um país. As métricas avaliadas incluem tanto fatores diretamente relacionados com a pegada de carbono de um país (como desmatamento) tanto como métricas mais gerais (como liberdade de imprensa), permitindo estabelecer relações diversas.

Como metodologia, aplicamos uma regressão linear sobre os diversos parâmetros, permitindo avaliar a relevância de cada um para o cálculo e criando um modelo de predição considerando mudanças nas métricas avaliadas.

## Métricas Utilizadas

Obtivemos as métricas de diversas fontes. Por isso, nem todos os dados estão disponíveis para todos os países avaliados ou para o mesmo ano. Utilizamos o dados mais recentes disponíveis em cada uma das fontes.

### Our World in Data ([fonte](https://ourworldindata.org/co2-emissions))

- Emissão de CO<sub>2</sub> em milhões de toneladas (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- PIB em dólares ajustdo para inflação de 2011 (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- População (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)

### ONU ([fonte](http://hdr.undp.org/en/indicators))

- **Índice de gini de distribuição de renda familiar (anos variados)**
- IDH (2019)
- Porcentagem da população residindo em áreas urbanas (2019)
- Porcentagem da variação na área de floresta (diferença entre 1990 e 2016)
- **Porcentagem da energia consumida proveniente de combustíveis fósseis (anos variados)**
- Porcentagem do Rendimento Nacional Bruto derivado da extração de recursos naturais (anos variados)
- Índice de desigualdade de gênero (2019)
- Expectativa de vida no nascimento (2019)
- **Porcentagem da população vivendo abaixo da linha da pobreza (anos variados)**
- **Porcentagem do PIB investido em pesquisa e desenvolvimento (anos variados)**
- **Proporção dos gastos públicos em educação e saúde sobre gastos militares (anos variados)**
- Porcentagem de importações e exportações sobre o PIB (anos variados)
- **Índice de desemprego (2019)**

### Center for Systemic Peace ([fonte](https://www.systemicpeace.org/polityproject.html))

- Polity Score, índice que avalia o nível de democracia de cada país - removemos países com valores especiais para evitar interferência na regressão (2018)

### Repórteres sem Fronteira ([fonte](https://rsf.org/en/ranking))

- Índice de liberdade de imprensa (2021)

### IMF ([fonte](https://www.imf.org/external/datamapper/NGDP_RPCH@WEO/OEMDC/ADVEC/WEOWORLD))

- Crescimento econômico (optamos pelos dados de 2019, já que 2020 representa um outlier)

### Banco Mundial ([fonte](https://datatopics.worldbank.org/what-a-waste/), [fonte](https://data.worldbank.org/indicator/EG.USE.PCAP.KG.OE))

- Resíduos sólidos urbanos gerados em toneladas por ano (anos variados)
- **Equivalente em kg de petróleo do consumo anual de energia (2013)**

### FAOSTAT ([fonte](http://www.fao.org/faostat/en/#data/QL))

- Produção de carne em toneladas (2016)

**Dados em negrito estão disponíveis para menos de 90% dos países selecionados**

In [1]:
import json
import csv

data = {}

co2_file_path = "owid-co2-data.json"
with open(co2_file_path, 'r', encoding='utf8') as json_file:
    co2_data = json.load(json_file)
    for key in co2_data.keys():
        if 'iso_code' in co2_data[key] and key != 'World':
            for i in range(len(co2_data[key]['data'])):
                if 'co2' in co2_data[key]['data'][i] and 'gdp' in co2_data[key]['data'][i] \
                        and 'population' in co2_data[key]['data'][i]:
                    data[co2_data[key]['iso_code']] = {
                        'name': key,
                        'co2_emissions': co2_data[key]['data'][i]['co2'],
                        'gdp': co2_data[key]['data'][i]['gdp'],
                        'population': co2_data[key]['data'][i]['population']               
                    }

X_fields = ['gdp', 'population']
                   
def get_iso3_from_country_name(country_name):
    iso3 = [key for key, values in data.items() if values['name'] == country_name]
    return iso3[0] if iso3 else None

def add_csv_data(file, country_row, data_row, data_name):
    X_fields.append(data_name)
    with open(file, encoding='utf8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [(data[key]['name'], key) for key in data]
        for line in reader:
            country_key = get_iso3_from_country_name(line[country_row])
            if country_key:
                data[country_key][data_name] = float(line[data_row])

def check_missing_countries(file, country_row):
    with open(file, encoding='utf8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [data[key]['name'] for key in data]
        file_names = [line[country_row] for line in reader]
        return [name for name in co2_names if name not in file_names]

add_csv_data('gini-index.csv', 1, -2, 'gini_index')
add_csv_data('hdi.csv', 1, -2, 'hdi')
add_csv_data('urban-population.csv', 1, -2, 'urban_population')
add_csv_data('forest-area-change.csv', 1, -2, 'forest_area_change')
add_csv_data('fossil-fuel-percentage.csv', 1, -2, 'fossil_fuel_percentage')
add_csv_data('natural-resource-depletion.csv', 1, -2, 'natural_resource_depletion')
add_csv_data('gender-inequality-index.csv', 1, -2, 'gender_inequality_index')
add_csv_data('life-expectancy.csv', 1, -2, 'life_expectancy')
add_csv_data('poverty-line.csv', 1, -2, 'poverty_line')
add_csv_data('research-and-development.csv', 1, -2, 'research_and_development')
add_csv_data('education-health-military-expenses.csv', 1, -2, 'education_health_military_expenses')
add_csv_data('exports-imports.csv', 1, -2, 'exports_imports')
add_csv_data('unemployment.csv', 1, -2, 'unemployment')
add_csv_data('democracy.csv', 4, 10, 'democracy')
add_csv_data('press-freedom.csv', 3, 7, 'press_freedom')
add_csv_data('economic-growth.csv', 0, -8, 'economic_growth')
add_csv_data('solid-waste.csv', 2, 26, 'solid_waste')
add_csv_data('meat-production.csv', 3, 11, 'meat_production')
add_csv_data('energy-use.csv', 0, -9, 'energy_use')

print('Países:', len(data.keys()))

Países: 164


## Pré-processamento dos Dados

Antes de executar a regressão, precisamos realizar algumas alterações nos dados para garantir um melhor resultado.

### Imputação de Dados Faltantes

Devido ao uso de diversas fontes de dados diferentes, muitas métricas não estão disponíveis para todos os países. Porém, o método de regressão linear não é capaz de lidar com valores nulos. Por isso, utilizamos um método simples de imputação de dados, onde valores nulos passam a ter o valor da média aritmética dos valores restantes. Essa atribuição tenta atribuir o valor que tenha o menor impacto possível no treinamento.

### Separação de Dados de Treinamento e Teste

Para validar nosso modelo, reservamos 10% dos países para teste no final e 90% para o treinamento e validações necessárias durante seu desenvolvimento.


### Normalização dos Dados

De modo a deixar as features na mesma escala, de 0 a 1, para melhorarmos a qualidade da regressão, fizemos a normalização dos dados.


In [13]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

countries = []
X = []
y = []
for values in data.values():
    countries.append(values['name'])
    y.append(values['co2_emissions'])
    X.append([values[field] if field in values else np.nan for field in X_fields])

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp.fit_transform(X)
y = np.array(y)

X_train_val, X_test, y_train_val, y_test, countries_train_val, countries_test \
        = train_test_split(X, y, countries, test_size=0.1, random_state=0)
X_train, X_val, y_train, y_val, countries_train, countries_val \
        = train_test_split(X_train_val, y_train_val, countries_train_val, test_size=0.1, random_state=0)


def normalize(array):
    min_by_feature = array.min(axis=0)
    max_by_feature = array.max(axis=0)
    
    normalized_array = array.copy()
    
    for j in range(0, len(array[0])):
        for i in range(0, len(array)):
            normalized_array[i, j] = (array[i, j] - min_by_feature[j]) / (max_by_feature[j] - min_by_feature[j])
    
    return normalized_array

# X_train = normalize(X_train)
# X_test = normalize(X_test)

# print(X_test)
# print(y_test)

print("Training countries:", countries_train)
print("Validation countries:", countries_val)
print("Testing countries:", countries_test)

Training countries: ['Equatorial Guinea', 'Canada', 'Mauritania', 'Iceland', 'Azerbaijan', 'Mali', 'Hong Kong', 'Peru', 'Belarus', 'Sri Lanka', 'Chad', 'Indonesia', 'Switzerland', 'Poland', 'Syria', 'Angola', 'Slovakia', 'Algeria', 'Bangladesh', 'Oman', 'Zambia', 'Pakistan', 'Georgia', 'United Arab Emirates', 'Paraguay', 'Portugal', 'Armenia', 'Hungary', 'Kuwait', 'Morocco', 'Ireland', 'Croatia', 'Vietnam', 'Cuba', 'Trinidad and Tobago', 'Rwanda', 'Sierra Leone', 'Serbia', 'Cape Verde', 'Kyrgyzstan', 'Saudi Arabia', 'Barbados', 'Palestine', 'Turkey', 'Burundi', 'Slovenia', 'Guinea-Bissau', 'Eswatini', 'Dominican Republic', 'Ukraine', 'Venezuela', 'United Kingdom', 'Uzbekistan', 'North Macedonia', 'Congo', 'Kenya', 'Germany', 'Romania', 'Denmark', 'Sao Tome and Principe', 'Namibia', 'Gambia', 'Greece', 'Guinea', 'Tanzania', 'Uganda', 'Botswana', 'Colombia', 'Netherlands', 'Myanmar', 'Argentina', 'Sweden', 'Libya', 'Sudan', 'China', 'Taiwan', 'Nigeria', 'Costa Rica', 'Mozambique', 'Cypru

### Decidindo a Regressão

Como passo inicial vamos testar vários modelos de regressão quem achamos adequado pela quantidade de dados que temos(um conjunto pequeno) utilizando os países do conjunto de treino e de validação e calcular seus respectivos erros de modo a escolher um melhor modelo para aplicar em nosso conjunto de teste.

Para calcular o erro do nosso modelo utilizaremos diretamente a média dos erros absolutos pois nos deu uma visão mais clara do resultado nos testes.

In [14]:
#RAplicando regressao linear
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_val)
errors = mean_absolute_error(y_val, y_pred)
print("Erro pela media absoluta: ", errors)
df = pd.DataFrame({'Países': countries_val, 'Atual': y_val, 'Predito': y_pred})
df

Erro pela media absoluta:  218.0043034128411


,Países,Atual,Predito
0,Togo,3.098,-110.928292
1,North Korea,24.621,-62.151530
2,Cambodia,9.724,-58.173057
3,Tajikistan,6.305,-37.405033
4,Nepal,8.841,-104.001589
5,Malta,1.399,-23.541334
6,Jamaica,8.141,-66.562705
7,Bosnia and Herzegovina,21.733,56.992024
8,Seychelles,0.594,-47.984740
9,India,2392.360,4761.899723


In [15]:
#Aplicando arvore de decisão
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
reg =  tree.DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_val)
errors = mean_absolute_error(y_val, y_pred)
print("Erro pela media absoluta: ", errors)
df = pd.DataFrame({'Países': countries_val, 'Atual': y_val, 'Predito': y_pred})
df

Erro pela media absoluta:  206.7576666666667


,Países,Atual,Predito
0,Togo,3.098,2.015
1,North Korea,24.621,9.554
2,Cambodia,9.724,9.905
3,Tajikistan,6.305,6.987
4,Nepal,8.841,6.745
5,Malta,1.399,7.368
6,Jamaica,8.141,6.987
7,Bosnia and Herzegovina,21.733,9.640
8,Seychelles,0.594,0.180
9,India,2392.360,5292.268


In [16]:
#Aplicando rede neural
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
reg =  MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
y_pred = reg.predict(X_val)
errors = mean_absolute_error(y_val, y_pred)
print("Erro pela media absoluta: ", errors)
df = pd.DataFrame({'Países': countries_val, 'Atual': y_val, 'Predito': y_pred})
df

Erro pela media absoluta:  9519982171.786802


,Países,Atual,Predito
0,Togo,3.098,-1.950677e+08
1,North Korea,24.621,-7.138000e+08
2,Cambodia,9.724,-8.719530e+08
3,Tajikistan,6.305,-4.890850e+08
4,Nepal,8.841,-1.244093e+09
5,Malta,1.399,-2.005258e+08
6,Jamaica,8.141,-3.528628e+08
7,Bosnia and Herzegovina,21.733,-6.928135e+08
8,Seychelles,0.594,-3.824016e+07
9,India,2392.360,-1.251484e+11


In [17]:
#Aplicando k vizinhos
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
#Fizemos testes com alguns valores de K e o melhor resultado se deu com k=3, portanto manteremos com esse numero
reg =  KNeighborsRegressor(n_neighbors=3).fit(X_train, y_train)
y_pred = reg.predict(X_val)
errors = mean_absolute_error(y_val, y_pred)
print("Erro pela media absoluta: ", errors)
df = pd.DataFrame({'Países': countries_val, 'Atual': y_val, 'Predito': y_pred})
df

Erro pela media absoluta:  88.89726666666668


,Países,Atual,Predito
0,Togo,3.098,1.986000
1,North Korea,24.621,11.066667
2,Cambodia,9.724,28.047000
3,Tajikistan,6.305,5.423333
4,Nepal,8.841,14.878333
5,Malta,1.399,1.986000
6,Jamaica,8.141,2.497667
7,Bosnia and Herzegovina,21.733,9.497667
8,Seychelles,0.594,0.341333
9,India,2392.360,1207.327000


### Escolhendo o modelo para nossos dados de teste

A maioria dos modelos apresentou erros próximos, exceto pelo modelo de redes neurais que acreditamos que nao seria o melhor modelo para uma regressão com poucos dados, por isso deve ter se perdido. Observando os resultados obtidos pelas regressões decidimos utilizar uma regressão pelo método de k vizinhos com k=3. Além do menor error com o conjunto de validação, escolhemos esse método pois o tamanho dos nossos dados não é grande e eles não tem um "formato" de função, ou seja, pode ter um pouco de "barulho" atrelado aos seus valores, por isso acreditamos que ao escolher k vizinhos com um valor não muito grande de k podemos ter o melhor resultado.

In [7]:
#Aplicando k vizinhos para nosso conjunto de teste
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
#Fizemos testes com alguns valores de K e o melhor resultado se deu com k=3, portanto manteremos com esse numero
reg =  KNeighborsRegressor(n_neighbors=3).fit(X_train, y_train)
y_pred = reg.predict(X_test)
errors = mean_absolute_error(y_test, y_pred)
print("Erro pela media absoluta: ", errors)
df = pd.DataFrame({'Países': countries_test, 'Atual': y_test, 'Predito': y_pred})
df

Erro pela media absoluta:  53.26452941176471


,Países,Atual,Predito
0,Thailand,281.705,235.356333
1,Malawi,1.263,2.767667
2,Nicaragua,5.448,5.423333
3,Austria,67.112,107.719333
4,Mauritius,4.346,6.406667
5,Ghana,16.547,21.178000
6,South Korea,617.960,468.406000
7,El Salvador,6.638,28.047000
8,Central African Republic,0.297,1.060667
9,Egypt,233.516,312.729667


### Conclusão

Conseguimos uma taxa de erro menor ainda com nossos dados de teste(e podemos ver pela tabela também) o que nos leva a acreditar que de fato escolhemos o melhor modelo dentre os considerados para realizar essa regressão.